In [19]:
import numpy as np
import pandas as pd
from confluent_kafka import Producer
from connect import get_connection
from random import randint
from simulation import transaktion_factory
from time import sleep
import socket


from confluent_kafka import Consumer

# Einlesen des Buchbestands

In [20]:
df_bestand = pd.read_csv('data/Bestand.csv', sep=';', encoding='unicode_escape')

In [21]:
bestand_cols=df_bestand.columns

# Einlesen der Tabellen des Producers

In [22]:
c = Consumer({
    'bootstrap.servers': '0.0.0.0:9092',
    'group.id': socket.gethostname(),
    'auto.offset.reset': 'latest'
})


In [23]:
prod_Transaktion_cols={
    0:'Aktion',
    1:'Datum',
    2:'ID_Exemplar',
    3:'ID_Kunde',
    4:'Titel',
    5:'Autor',
    6:'Jahr',
    7:'Art',
    8:'Kennung',
    9:'Zugriffsort'
}

prod_Bewertung_cols={
    0:'Aktion',
    1:'ID_Kunde',
    2:'ID_Buch',
    3:'Wertung',
    4:'Rezension'
}

prod_Neukunden_cols={
    0:'Aktion',
    1:'ID_Kunde',
    2:'Kundennr',
    3:'Vorname_1',
    4:'Vorname_2',
    5:'Nachname',
    6:'Anrede',
    7:'PLZ',
    8:'Strasse',
    9:'Hausnr',
   10:'Mail',
   11:'Tel',
   12:'Geschlecht',
   13:'Geburtsdatum',
   14:'Beruf',
   15:'Titel',
   16:'PersoNr',
   17:'PersoValidTo' 
}

cols=[prod_Transaktion_cols,prod_Bewertung_cols,prod_Neukunden_cols]
topics=["Transaktion", "Bewertung", "Neukunden"]

dfd={}

In [24]:
for col, topic in zip(cols, topics):
    print(col, topic)

    dfd[topic] = pd.DataFrame(columns=list(col.values()))
    
    
    c.subscribe([topic])
    
    for i in range(60):
        msg = c.poll(1)
        if msg is None:
            continue
    
        mrow = pd.DataFrame([msg.value().decode('utf-8').split(";")]).rename(columns=col)

        dfd[topic]=pd.concat([dfd[topic], mrow])

    dfd[topic] = dfd[topic].reset_index(drop=True)
    #c.close()

{0: 'Aktion', 1: 'Datum', 2: 'ID_Exemplar', 3: 'ID_Kunde', 4: 'Titel', 5: 'Autor', 6: 'Jahr', 7: 'Art', 8: 'Kennung', 9: 'Zugriffsort'} Transaktion
{0: 'Aktion', 1: 'ID_Kunde', 2: 'ID_Buch', 3: 'Wertung', 4: 'Rezension'} Bewertung
{0: 'Aktion', 1: 'ID_Kunde', 2: 'Kundennr', 3: 'Vorname_1', 4: 'Vorname_2', 5: 'Nachname', 6: 'Anrede', 7: 'PLZ', 8: 'Strasse', 9: 'Hausnr', 10: 'Mail', 11: 'Tel', 12: 'Geschlecht', 13: 'Geburtsdatum', 14: 'Beruf', 15: 'Titel', 16: 'PersoNr', 17: 'PersoValidTo'} Neukunden


In [25]:
dfd['Transaktion']

,Aktion,Datum,ID_Exemplar,ID_Kunde,Titel,Autor,Jahr,Art,Kennung,Zugriffsort
0,Leihe,2024-12-08 16:00:00,230,3460,Der Messias,Friedrich Gottlieb Klopstock,1748.0,Versepos,Mvc31862,Czw27103
1,Rückgabe,2024-12-08 20:00:00,182,5752,Die Kartause von Parma,Stendhal,1839.0,Roman,HFs71093,leQ41379
2,Leihe,2024-12-08 22:00:00,131,1643,Ragtime,Edgar Lawrence Doctorow,1975.0,Roman,Izl93154,OSY68140
3,Rückgabe,2024-12-09 02:00:00,47,6270,Anleitung zum Unglücklichsein,Paul Watzlawick,1983.0,Sachbuch,fBa85403,Pox83067
4,Leihe,2024-12-09 04:00:00,130,1707,Das große Protokoll gegen Zwetschkenbaum,Albert Drach,1964.0,Roman,lsX82514,snu06253
5,Leihe,2024-12-09 07:00:00,142,2126,Haben oder Sein,Erich Fromm,1976.0,Philosophie,Ufi42038,lbP63102
6,Leihe,2024-12-09 10:00:00,114,359,med ana schwoazzn dintn,Hans Carl Artmann,1958.0,Lyrik,Djd28756,KxD97842
7,Leihe,2024-12-09 11:00:00,171,6680,Der Medicus,Noah Gordon,1986.0,Roman,ULl58743,gbU15348
8,Leihe,2024-12-09 11:00:00,49,3013,Gegen den Strich,Joris-Karl Huysmans,1884.0,Roman,DZk37249,wuI63491
9,Leihe,2024-12-09 13:00:00,179,6957,Als wär's ein Stück von mir,Carl Zuckmayer,1966.0,Autobiografie,FjJ76904,yEF38051


In [26]:
dfd['Bewertung']

,Aktion,ID_Kunde,ID_Buch,Wertung,Rezension
0,Bewertung,148,274,5,Das Buch war weder besonders beeindruckend no...
1,Bewertung,216,431,7,Es gibt einige brillante Momente in diesem Bu...
2,Bewertung,123,989,3,Verfehlt die Erwartungen.
3,Bewertung,180,955,4,Trotz einiger interessanter Ansätze konnte da...
4,Bewertung,144,786,9,Ein echter Pageturner!
5,Bewertung,209,327,9,"Ein nahezu makelloses Werk, das durch Tiefe u..."
6,Bewertung,131,312,7,Bietet solide Unterhaltung.
7,Bewertung,95,840,6,Nett für zwischendurch.
8,Bewertung,119,387,9,Einfach grandios!
9,Bewertung,194,177,3,Das Buch ließ viel zu wünschen übrig. Die Erz...


In [27]:
dfd['Neukunden']

,Aktion,ID_Kunde,Kundennr,Vorname_1,Vorname_2,Nachname,Anrede,PLZ,Strasse,Hausnr,Mail,Tel,Geschlecht,Geburtsdatum,Beruf,Titel,PersoNr,PersoValidTo
0,Neukunde,303,4024,Jonas,nan,Meier,Herr,23974,Jahnstraße,2.0,jonas.meier@gmail.com,038426/50909,Männlich,1952-07-06,Senior/in,None,HSC25048,2023-11-07
1,Neukunde,304,2620,Lena,nan,Richter,Frau,56242,Am Hang,8.0,lena.richter@gmx.net,02626/886119,W,1960-09-11,Sektions- und Präparationsassistent/in,None,LPU21345,04.11.2021
2,Neukunde,305,6024,Sofia,nan,Schulz,Frau,99735,Neue Straße,6.0,sofia.schulz@web.de,036334/493437,W,14.11.1933,Senior/in,None,MET24518,Sat Aug 23 2025
3,Neukunde,306,4071,Lena,nan,Schulze,Frau,49393,Ringstraße,5.0,lena.schulze@t-online.de,04442/634449,w,1926-09-16,Senior/in,None,FSD89507,2022-11-29
4,Neukunde,307,4020,Niklas,nan,Werner,Herr,74749,Ringstraße 6,nan,niklas.werner@gmail.com,06295/766117,m,Mon Apr 20 1970,Key Account Manager/in,None,JFX96450,24.03.2023
5,Neukunde,308,3678,Lara,nan,Schwarz,Frau,18196,Lindenstraße,4.0,lara.schwarz@t-online.de,038208/830473,w,Fri Dec 01 1950,Senior/in,Dr.,TNO56790,06.06.2021
6,Neukunde,309,6319,Julian,nan,Müller,Hr.,16230,Brunnenstraße,3.0,julian.müller@web.de,033451/527927,M,26/05/1986,Tiermedizinische/r Fachangestellte/r,None,VIR83619,2021-06-07
7,Neukunde,310,3896,Henry,nan,Neumann,Herr,19322,Kastanienweg,5.0,henry.neumann@gmx.net,038787/282065,m,01.09.1928,Senior/in,Dr.,ZCU91628,2023-08-31
8,Neukunde,311,6748,Tim,nan,Müller,Herr,16792,Industriestraße,8.0,tim.müller@t-online.de,03307/116473,M,2007-04-23,Business Analyst,None,QXW94028,2022-05-15
9,Neukunde,312,5881,Philipp,nan,Maier,Herr,1097,Burgstraße,10.0,philipp.maier@gmail.com,0351/602880,m,1987-10-27,Verwaltungsfachangestellte/r,None,WRZ34970,2020-01-08


# Cleaning der Tabellen

### Transaktion

In [28]:
# IDs zu int konvertieren
dfd['Transaktion']['ID_Exemplar'] = pd.to_numeric(dfd['Transaktion']['ID_Exemplar']).astype(int)
dfd['Transaktion']['ID_Kunde'] = pd.to_numeric(dfd['Transaktion']['ID_Kunde']).astype(int)
# Jahre zu int konvertieren
dfd['Transaktion']['Jahr'] = pd.to_numeric(dfd['Transaktion']['Jahr']).astype(int)

### Bewertung

In [29]:
# IDs zu int konvertieren
dfd['Bewertung']['ID_Kunde'] = pd.to_numeric(dfd['Bewertung']['ID_Kunde']).astype(int)
dfd['Bewertung']['ID_Buch'] = pd.to_numeric(dfd['Bewertung']['ID_Buch']).astype(int)
dfd['Bewertung']['Wertung'] = pd.to_numeric(dfd['Bewertung']['Wertung']).astype(int)

### Neukunden

In [30]:
# IDs zu int konvertieren
dfd['Neukunden']['ID_Kunde'] = pd.to_numeric(dfd['Neukunden']['ID_Kunde']).astype(int)
dfd['Neukunden']['Kundennr'] = pd.to_numeric(dfd['Neukunden']['Kundennr']).astype(int)

# Anreden konsistent machen 
dfd['Neukunden']['Anrede'] = dfd['Neukunden']['Anrede'].replace(['Fr\.'],'Frau', regex=True)
dfd['Neukunden']['Anrede'] = dfd['Neukunden']['Anrede'].replace(['Hr\.'],'Herr', regex=True)
# Titel: "None" to NaN
dfd['Neukunden']['Titel']  = np.where(dfd['Neukunden']['Titel'].str.contains("None"), np.nan, dfd['Neukunden']['Titel'] )
# Vorname_2: "nan" to NaN
dfd['Neukunden']['Vorname_2']  = np.where(dfd['Neukunden']['Vorname_2'].str.contains("nan"), np.nan, dfd['Neukunden']['Vorname_2'] )
# Hausnr: "nan" to NaN
dfd['Neukunden']['Hausnr']  = np.where(dfd['Neukunden']['Hausnr'].str.contains("nan"), np.nan, dfd['Neukunden']['Hausnr'] )
# 4-stellige PLZ mit "0" präfixen 
dfd['Neukunden']['PLZ']    = dfd['Neukunden']['PLZ'].str.strip().str.rjust(5, '0')

# Hausnummern aus Strasse-Spalte extrahieren
def extract_hausnr(strasse):
  if pd.isna(strasse):
    return np.nan
  parts = strasse.split()
  if len(parts) > 1 and parts[-1].isdigit():
    return parts[-1]
  else:
    return np.nan
#Apply extract_hausnr, wenn 'Hausnr' NaN und entferne die Hausnummern aus der Strasse Spalte
dfd['Neukunden'].loc[dfd['Neukunden']['Hausnr'].isnull(), 'Hausnr'] = dfd['Neukunden'].loc[dfd['Neukunden']['Hausnr'].isnull(), 'Strasse'].apply(extract_hausnr)
dfd['Neukunden']['Strasse'] = dfd['Neukunden']['Strasse'].str.rsplit(expand=True)[0]
# Convert Hausnr to int
dfd['Neukunden']['Hausnr'] = pd.to_numeric(dfd['Neukunden']['Hausnr']).astype(int)

# Geschlecht konsistent machen 
dfd['Neukunden']['Geschlecht'] = dfd['Neukunden']['Geschlecht'].str.strip().replace(["^w$", "^W$"],'Weiblich', regex=True)
dfd['Neukunden']['Geschlecht'] = dfd['Neukunden']['Geschlecht'].str.strip().replace(["^m$", "^M$"],'Männlich', regex=True)

# Datumsspalten konsistent machen
dfd['Neukunden']['Geburtsdatum'] = pd.to_datetime(dfd['Neukunden']['Geburtsdatum'],format='mixed', dayfirst=True)
dfd['Neukunden']['PersoValidTo'] = pd.to_datetime(dfd['Neukunden']['PersoValidTo'],format='mixed', dayfirst=True)

### Bestand

In [31]:
# Jahre zu int konvertieren
df_bestand['Jahr'] = pd.to_numeric(df_bestand['Jahr']).astype('Int64', errors='ignore')

# Aufbau der Tabellen für das Core DWH

In [32]:
dfs={}

In [33]:
Kunde_cols={
    0:'ID_Kunde',
    1:'ID_Mitgliedsstatus',
    2:'Kundennr',
    3:'Vorname_1',
    4:'Vorname_2',
    5:'Nachname',
    6:'Anrede',
    7:'Titel',
    8:'PLZ',
    9:'Strasse',
   10:'Hausnr',
   11:'Mail',
   12:'Tel',
   13:'Geschlecht',
   14:'Geburtsdatum',
   15:'Beruf',
   16:'PersoNr',
   17:'PersoValidTo', 
   18:'Mitglied seit',
   19:'Mitglied bis' 
}

Bewertung_cols={
    1:'ID_Kunde',
    2:'ID_Buch',
    3:'Wertung',
    4:'Rezension'
}

Buch_cols={
    1:'ID_Kunde',
    2:'ID_Buch',
    3:'Wertung',
    4:'Rezension'
}

Exemplare_cols={
    1:'ID_Bestand',
    2:'Nr',
    3:'Kennung',
    4:'Zugriffsort',
    5:'Zustand'
}

Buch_cols={
    1:'ID_Autor',
    2:'Nr',
    3:'Titel',
    4:'Jahr',
    5:'Art',
}

Leihe_cols={
    0:'ID_Kunde',
    1:'ID_Exemplar',
    2:'Ausleihdatum',
    3:'Rueckgabedatum',
    4:'Verlaengerungsstatus',
    5:'Mahnstatus',
    6:'Fernleihe'
}

Mitgliedsstatus_cols={
    0:'ID_Mitgliedsstatus',
    2:'Bezeichnung',
    3:'Jahresbeitrag',
    4:'Mahnbetrag'
}

Beitragszahlung_cols={
    0:'ID_Zahlung',
    2:'ID_Kunde',
    3:'Datum',
    4:'Betrag',
}

In [34]:
dfs['Kunde']     = dfd['Neukunden'].reindex(columns=list(Kunde_cols.values()))
dfs['Bewertung'] = dfd['Bewertung'].reindex(columns=list(Bewertung_cols.values()))
dfs['Exemplare'] = df_bestand.reindex(columns=list(Exemplare_cols.values())).rename(columns={
                                                                        'ID_Bestand':'ID_Exemplar',
                                                                        'Nr':'ID_Buch'    
                                                                        })

dfs['Autor']   = df_bestand[['Autor', 'Herkunft']].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'ID_Autor'})

dfs['Buch']    = pd.merge(df_bestand, 
                          dfs['Autor'], 
                          on=['Autor']
                         )[list(Buch_cols.values())]                                  \
                                                    .drop_duplicates()                \
                                                    .reset_index(drop=True)           \
                                                    .rename(columns={'Nr':'ID_Buch'})

dfs['Leihe'] = dfd['Transaktion'].reindex(columns=list(Leihe_cols.values())).reset_index().rename(columns={'index':'ID_Leihe'})


### more ###
dfs['Mitgliedsstatus'] = pd.DataFrame(columns=list(Mitgliedsstatus_cols.values()))
dfs['Beitragszahlung'] = dfd['Neukunden'].reindex(columns=list(Beitragszahlung_cols.values()))


%4|1727339292.849|MAXPOLL|rdkafka#consumer-2| [thrd:main]: Application maximum poll interval (300000ms) exceeded by 453ms (adjust max.poll.interval.ms for long-running message processing): leaving group


# Schicke die Tabellen an den SQL Server


In [36]:
from sqlalchemy import create_engine, inspect, text

USER = 'root'
PASSWORD = 'root'
HOST = 'localhost'          # see yml file
PORT = 3307
DATABASE = 'test_db_1'
 

def get_connection(database=DATABASE):
    return create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}"\
            .format(USER, PASSWORD, HOST, PORT, database)
    )

engine = get_connection('mysql')
with engine.connect() as connection:
    insp = inspect(engine)
    db_list = insp.get_schema_names()
    print(f"Connection to the {HOST} for user {USER} created successfully.")
    if DATABASE not in db_list:
        sql = text(f"CREATE DATABASE {DATABASE} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
        result = connection.execute(sql)
        if result:
            print(f"Database {DATABASE} created!")

Connection to the localhost for user root created successfully.


In [37]:
for Tabelle in dfs.keys():
    dfs[Tabelle].to_sql(Tabelle, con=engine,schema=DATABASE, index=False, if_exists='append') 